# _**Title: Feature Engineering for Fashion Retail Sales Analysis**_

## 1. Introduction
This notebook focuses on feature engineering to prepare the cleaned dataset for predictive modeling and deeper insights into customer behavior. 
We will transform numerical features, extract time-based features, encode categorical variables, and create customer-level aggregates for further analysis.

In [5]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Load processed dataset
df = pd.read_csv("../data/processed/retail_cleaned_eda.csv")
# initial overview
print("Dataset Loaded Successfully!")
print("Shape of the dataset:", df.shape)
df.head()

Dataset Loaded Successfully!
Shape of the dataset: (3400, 7)


,customer_reference_id,item_purchased,purchase_amount_(usd),date_purchase,review_rating,payment_method,log_purchase_amount
0,4018,Handbag,4619.0,2023-02-05,3.2,Credit Card,8.438150
1,4115,Tunic,2456.0,2023-07-11,2.0,Credit Card,7.806696
2,4019,Tank Top,2102.0,2023-03-23,4.1,Cash,7.651120
3,4097,Leggings,3126.0,2023-03-15,3.2,Cash,8.047829
4,3997,Wallet,3003.0,2022-11-27,4.7,Cash,8.007700


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_reference_id  3400 non-null   int64  
 1   item_purchased         3400 non-null   object 
 2   purchase_amount_(usd)  3400 non-null   float64
 3   date_purchase          3400 non-null   object 
 4   review_rating          3400 non-null   float64
 5   payment_method         3400 non-null   object 
 6   log_purchase_amount    3400 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 186.1+ KB
